In [0]:
# Create widgets
dbutils.widgets.text("catalog_val", "system", "Catalog")
dbutils.widgets.text("target_catalog_val", "tagetik", "Target Catalog")
dbutils.widgets.text("schema_val", "raw", "Schema")
dbutils.widgets.text("table_val", "dsl", "Table")
dbutils.widgets.text("column_val", "entity", "Column")
dbutils.widgets.text("email_val", "lorenzo.nizatti@met.coom", "Email")

In [0]:
catalog_val = dbutils.widgets.get("catalog_val")
target_catalog_val = dbutils.widgets.get("target_catalog_val")
schema_val = dbutils.widgets.get("schema_val")
table_val = dbutils.widgets.get("table_val")
column_val = dbutils.widgets.get("column_val")
email_val = dbutils.widgets.get("email_val")

table_full_name_val = f"{target_catalog_val}.{schema_val}.{table_val}"


In [0]:
query = f"""
SELECT
  mask(created_by) as created_by,
  entity_type,
  source_type,
  COUNT(distinct event_time) as access_count,
  MIN(event_date) as first_access_date,
  MAX(event_date) as last_access_date
FROM
  `{table_full_name_val}`.access.table_lineage
WHERE
  source_table_catalog = '{catalog_val}'
  AND source_table_schema = '{schema_val}'
  AND source_table_name = '{table_val}'
  AND datediff(now(), event_date) < 7
  AND entity_type IS NOT NULL
  AND source_type IS NOT NULL
GROUP BY
  entity_type, source_type
ORDER BY
  access_count DESC
"""

display(spark.sql(query))

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-3123072426457900>, line 24
      1 query = f"""
      2 SELECT
      3   mask(created_by) as created_by,
   (...)
     21   access_count DESC
     22 """
---> 24 display(spark.sql(query))

File /databricks/spark/python/pyspark/instrumentation_utils.py:47, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     45 start = time.perf_counter()
     46 try:
---> 47     res = func(*args, **kwargs)
     48     logger.log_success(
     49         module_name, class_name, function_name, time.perf_counter() - start, signature
     50     )
     51     return res

File /databricks/spark/python/pyspark/sql/session.py:1748, in SparkSession.sql(self, sqlQuery, args, **kwargs)
   1744         assert self._jvm is not None
   1745         litArgs = self._jvm.PythonUtils.toArray(
   1746             [_to_java_column(lit(v)) for v in (args